# Imports

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

import numpy as np
import requests as rq
import io, h5py

# Retrieve Dataset

In [2]:
data = rq.get('https://www.dropbox.com/s/c3umbo5y13sqcfp/synthetic_dataset.h5?raw=true')
data.raise_for_status()

with h5py.File(io.BytesIO(data.content), 'r') as dataset:
    x_train = np.array(dataset['X_train']).astype(np.float32).transpose([0, 2, 1])
    y_train = np.array(dataset['Y_train']).astype(np.float32)
    x_valid = np.array(dataset['X_valid']).astype(np.float32).transpose([0, 2, 1])
    y_valid = np.array(dataset['Y_valid']).astype(np.int32)
    x_test = np.array(dataset['X_test']).astype(np.float32).transpose([0, 2, 1])
    y_test = np.array(dataset['Y_test']).astype(np.int32)

print(x_train.shape, y_train.shape)

(21000, 200, 4) (21000, 12)


# Connect to Drive (Optional)

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Construct Model

In [3]:
def build_model(params):
    # Hyperparameters
    filters, kernel_size, batch_norm_1, pool_size, dropout_1, pos_encoding, heads, key_size, dropout_2, dense_size, batch_norm_2, dropout_3 = params
    
    # Input Layer
    inputs = layers.Input(shape=(200, 4))
    
    # Convolution Block
    nn = layers.Conv1D(filters=filters, kernel_size=kernel_size, use_bias=False, padding='same')(inputs)
    if batch_norm_1:
        nn = layers.BatchNormalization()(nn)
    nn = layers.Activation('relu')(nn)
    nn = layers.MaxPool1D(pool_size=pool_size)(nn)
    if dropout_1 != 0:
        nn = layers.Dropout(dropout_1)(nn)
    
    # Multi-Head Attention Block
    if pos_encoding:
        positions = tf.range(nn.shape[1])
        context = layers.Embedding(input_dim=nn.shape[1], output_dim=nn.shape[2])(positions)
        nn = tf.add(nn, context)  # contextual meaning

    attention, weights = layers.MultiHeadAttention(num_heads=heads, key_dim=key_size)(nn, nn, return_attention_scores=True)
    if dropout_2 != 0:
        nn = layers.Dropout(dropout_2)(attention)
    nn = layers.LayerNormalization()(nn)
    
    # Dense Block
    nn = layers.Flatten()(nn)
    nn = layers.Dense(dense_size, use_bias=False)(nn)
    if batch_norm_2:
        nn = layers.BatchNormalization()(nn)
    nn = layers.Activation('relu')(nn)
    if dropout_3 != 0:
        nn = layers.Dropout(dropout_3)(nn)
    
    # Outputs
    outputs = layers.Dense(12, activation='sigmoid')(nn)
    
    # Build Model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0005), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(curve='ROC', name='auroc'), tf.keras.metrics.AUC(curve='PR', name='aupr')])
    return model

In [7]:
# filters, kernel_size, batch_norm_1, pool_size, dropout_1, pos_encoding, heads, key_size,
# dropout_2, dense_size, batch_norm_2, dropout_3
base1 = [32, 19, True, 4, 0.1, False, 8, 64, 0.1, 512, True, 0.5]
base2 = [32, 19, True, 4, 0.1, False, 8, 64, 0.1, 512, True, 0.5]

# Alternate parameters
filters = [32, 128, 256]
kernels = [19]
batch_norm_1 = [True, False]
pools = [1, 4, 10, 20]
dropout_1 = [0.1, 0.5]
pos_encodings = [True, False]
heads = [1, 8, 16]
keys = [32, 64, 128, 256]
dropout_2 = [0.1, 0.5]
denses = [64, 256, 512]
batch_norm_2 = [True, False]
dropout_3 = [0.1, 0.5]

names = ['filters', 'kernels', 'batch_norm_1', 'pools', 'dropout_1', 'pos_encodings', 'heads', 'keys', 'dropout_2', 'denses', 'batch_norm_2', 'dropout_3']
params = [filters, kernels, batch_norm_1, pools, dropout_1, pos_encodings, heads, keys, dropout_2, denses, batch_norm_2, dropout_3]

model = build_model(base1)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_aupr', patience=15, verbose=1, mode='max', restore_best_weights=False)
lr_decay = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_aupr', factor=0.2, patience=5, min_lr=1e-7, mode='max', verbose=1) 
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=f'/content/drive/MyDrive/model-test')

model.fit(x=x_train, y=y_train, epochs=75, validation_data=(x_valid, y_valid), callbacks=[early_stop, lr_decay, checkpoint])

"""for i in range(len(params)):
    for j in range(len(params[i])):
        direc = names[i]
        name = f'model-{params[i][j]}'
        
        args = base1.copy()
        args[i] = params[i][j]
        
        model = build_model(args)
        
        # Callbacks
        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_aupr', patience=15, verbose=1, mode='max', restore_best_weights=False)
        lr_decay = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_aupr', factor=0.2, patience=5, min_lr=1e-7, mode='max', verbose=1) 
        tensorboard = tf.keras.callbacks.Tensorboard(log_dir=f'/content/drive/MyDrive/ColabNotebooks/ConvAttTests/logs/{direc}/{name}')
        checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=f'/content/drive/MyDrive/ColabNotebooks/ConvAttTets/models/{direc}/{name}')
        
        model.fit(x=x_train, y=y_train, epochs=10, validation_data=(x_valid, y_valid), callbacks=[early_stop, lr_decay, tensorboard, checkpoint])"""

Epoch 1/75
657/657 [==============================] - 7s 9ms/step - loss: 0.5206 - auroc: 0.5343 - aupr: 0.1696 - val_loss: 0.5681 - val_auroc: 0.6641 - val_aupr: 0.3325


INFO:tensorflow:Assets written to: /content/drive/MyDrive/model-test/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/model-test/assets


Epoch 2/75
657/657 [==============================] - 5s 8ms/step - loss: 0.3503 - auroc: 0.7452 - aupr: 0.4594 - val_loss: 0.3471 - val_auroc: 0.7959 - val_aupr: 0.5188


INFO:tensorflow:Assets written to: /content/drive/MyDrive/model-test/assets


...

Epoch 53/75
657/657 [==============================] - 5s 8ms/step - loss: 0.1603 - auroc: 0.9578 - aupr: 0.8687 - val_loss: 0.1629 - val_auroc: 0.9619 - val_aupr: 0.8701

Epoch 00053: ReduceLROnPlateau reducing learning rate to 8.000000889296644e-07.


INFO:tensorflow:Assets written to: /content/drive/MyDrive/model-test/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/model-test/assets


Epoch 00053: early stopping


"for i in range(len(params)):\n    for j in range(len(params[i])):\n        direc = names[i]\n        name = f'model-{params[i][j]}'\n        \n        args = base1.copy()\n        args[i] = params[i][j]\n        \n        model = build_model(args)\n        \n        # Callbacks\n        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_aupr', patience=15, verbose=1, mode='max', restore_best_weights=False)\n        lr_decay = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_aupr', factor=0.2, patience=5, min_lr=1e-7, mode='max', verbose=1) \n        tensorboard = tf.keras.callbacks.Tensorboard(log_dir=f'/content/drive/MyDrive/ColabNotebooks/ConvAttTests/logs/{direc}/{name}')\n        checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=f'/content/drive/MyDrive/ColabNotebooks/ConvAttTets/models/{direc}/{name}')\n        \n        model.fit(x=x_train, y=y_train, epochs=10, validation_data=(x_valid, y_valid), callbacks=[early_stop, lr_decay, tensorboard, checkpoint])"